In [ ]:
# %%
import torch
torch.cuda.is_available()


True

In [ ]:
# %%
# (Colab only) upload your XLSX
from google.colab import files
uploaded = files.upload()


Saving evrensel_isci_sendika_2024_dec2025_clean_fin_uncorrupted_real.xlsx to evrensel_isci_sendika_2024_dec2025_clean_fin_uncorrupted_real.xlsx


In [ ]:
# %%
!pip -q install transformers accelerate evaluate openpyxl scikit-learn pandas numpy torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.4 MB/s eta 0:00:00


In [ ]:
# %%
# If you haven't installed these in this environment, uncomment
!pip -q install "transformers>=4.38"

import re
import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE


'cuda'

In [ ]:
# %%
PATH_XLSX = "evrensel_isci_sendika_2024_dec2025_clean_fin_uncorrupted_real.xlsx"  # <-- change this
df = pd.read_excel(PATH_XLSX)

required_cols = ["EVENT_RELEVANT", "EVENT_ID", "title", "content","date"]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns in XLSX: {missing}")
TITLE_COL = "title"
CONTENT_COL = "content"
TEXT_COL = "text"
DATE_COL = "date"     # or "date" if you use that
MANUAL_COL = "EVENT_RELEVANT"   # manual labels (sparse), can be missing
EVENT_ID_COL = "EVENT_ID"      # may exist or be empty/NaN
EVENT_ID = EVENT_ID_COL

df.shape, df.columns.tolist()


((9186, 15),
 ['title',
  'date',
  'link',
  'content',
  'EVENT_RELEVANT',
  'EVENT_ID',
  'Unnamed: 6',
  'error',
  'Unnamed: 8',
  'Unnamed: 9',
  'Unnamed: 10',
  'Unnamed: 11',
  'Unnamed: 12',
  650,
  160])

In [ ]:
# %%
def normalize_text(x):
    """Light cleanup: keep Turkish characters, remove weird spaces, collapse whitespace."""
    if pd.isna(x):
        return ""
    x = str(x).replace("\u00A0", " ")  # non-breaking space
    x = re.sub(r"\s+", " ", x).strip()
    return x

df["title"] = df["title"].apply(normalize_text)
df["content"] = df["content"].apply(normalize_text)

# final text fed into the model
df["text"] = (df["title"].astype(str) + "\n\n" + df["content"].astype(str)).str.strip()
df[["title","content","text"]].head(2)


,title,content,text
0,Bartın'da Hema'ya ait maden ocağında vagonları...,Bartın'ın Amasra ilçesindeki Hema Enerji şirke...,Bartın'da Hema'ya ait maden ocağında vagonları...
1,Bu soygun düzeni değişmeli,Pendik Marmara Eğitim ve Araştırma Hastanesind...,Bu soygun düzeni değişmeli\n\nPendik Marmara E...


In [ ]:
# %%
# ----------------------------
# Cell 4 (fixed) — Define labeled rows (accept 0/1 as floats OR strings)
# ----------------------------
import numpy as np
import pandas as pd

def normalize_label(x):
    if pd.isna(x):
        return np.nan

    # float/integer case (your current situation: 0.0 / 1.0)
    if isinstance(x, (int, np.integer, float, np.floating)):
        if x == 0 or x == 0.0:
            return 0
        if x == 1 or x == 1.0:
            return 1
        return np.nan

    # string case
    s = str(x).strip().lower()
    if s in {"0", "0.0", "no", "n", "false"}:
        return 0
    if s in {"1", "1.0", "yes", "y", "true"}:
        return 1

    return np.nan

df["LABEL_CLEAN"] = df["EVENT_RELEVANT"].apply(normalize_label)
labeled_mask = df["LABEL_CLEAN"].notna()

print("Total rows:", len(df))
print("Labeled rows:", int(labeled_mask.sum()))
df.loc[labeled_mask, ["EVENT_RELEVANT","LABEL_CLEAN"]].head(10)


Total rows: 9186
Labeled rows: 738


,EVENT_RELEVANT,LABEL_CLEAN
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
5,0.0,0.0
6,0.0,0.0
7,0.0,0.0
8,0.0,0.0
9,0.0,0.0


In [ ]:
# %%
# ----------------------------
# Cell 5 — Train/validation split (using LABEL_CLEAN from Cell 4)
# ----------------------------

# Keep only labeled rows (LABEL_CLEAN is 0/1, NaN otherwise)
df_labeled = df.loc[labeled_mask].copy()

# This is what the Trainer will learn on
df_labeled["label"] = df_labeled["LABEL_CLEAN"].astype(int)

# Stratified split so class balance is preserved in train/val
train_df, val_df = train_test_split(
    df_labeled,
    test_size=0.2,
    random_state=42,
    stratify=df_labeled["label"]
)

print("Train size:", len(train_df), "Val size:", len(val_df))
train_df["label"].value_counts(normalize=True), val_df["label"].value_counts(normalize=True)


Train size: 590 Val size: 148


(label
 0    0.783051
 1    0.216949
 Name: proportion, dtype: float64,
 label
 0    0.783784
 1    0.216216
 Name: proportion, dtype: float64)

In [ ]:
# %%
# Inspect what EVENT_RELEVANT really looks like
s = df["EVENT_RELEVANT"]

print("dtype:", s.dtype)
print("non-null count:", s.notna().sum())

# show a sample of unique raw values (as-is)
u = s.dropna().unique()
print("unique values sample (up to 50):", u[:50])

# show stringified + stripped sample too
u_str = pd.Series(u).astype(str).str.strip()
print("stringified sample (up to 50):", u_str.head(50).tolist())


dtype: float64
non-null count: 738
unique values sample (up to 50): [0. 1.]
stringified sample (up to 50): ['0.0', '1.0']


In [ ]:
# %%
MODEL_NAME = "dbmdz/bert-base-turkish-cased"  # BERTurk
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

MAX_LEN = 384  # 512 is allowed but slower

class TextClsDataset(Dataset):
    def __init__(self, texts, labels=None, tokenizer=None, max_len=384):
        self.texts = list(texts)
        self.labels = None if labels is None else list(labels)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, i):
        enc = self.tokenizer(
            self.texts[i],
            truncation=True,
            max_length=self.max_len,
            padding=False
        )
        item = {k: torch.tensor(v) for k, v in enc.items()}
        if self.labels is not None:
            item["labels"] = torch.tensor(int(self.labels[i]))
        return item

train_ds = TextClsDataset(train_df["text"], train_df["label"], tokenizer, MAX_LEN)
val_ds   = TextClsDataset(val_df["text"],   val_df["label"],   tokenizer, MAX_LEN)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

In [ ]:
# %%
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2).to(DEVICE)

pos = int(train_df["label"].sum())
neg = int(len(train_df) - pos)

# More weight to positive class if positives are rare
class_weights = torch.tensor([1.0, (neg / max(pos, 1))], dtype=torch.float, device=DEVICE)
class_weights


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([1.0000, 3.6094], device='cuda:0')

In [ ]:
# %%
# ----------------------------
# Cell 8 (updated) — Trainer setup + fine-tune (version compatible)
# ----------------------------
import inspect

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = torch.softmax(torch.tensor(logits), dim=-1).numpy()[:, 1]
    return {
        "roc_auc": float(roc_auc_score(labels, probs)) if len(np.unique(labels)) > 1 else float("nan")
    }

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Weighted loss
def custom_loss_fn(model, inputs, return_outputs=False):
    labels = inputs.pop("labels")
    outputs = model(**inputs)
    logits = outputs.logits
    loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
    loss = loss_fct(logits, labels)
    return (loss, outputs) if return_outputs else loss

# Some transformers versions support compute_loss in Trainer; fallback safely
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        return custom_loss_fn(model, inputs, return_outputs=return_outputs)

args = TrainingArguments(
    output_dir="berturk_event_detect",
    learning_rate=2e-5,
    per_device_train_batch_size=8 if DEVICE=="cuda" else 4,
    per_device_eval_batch_size=16 if DEVICE=="cuda" else 8,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="roc_auc",
    greater_is_better=True,
    fp16=(DEVICE=="cuda"),
    report_to=[]
)

trainer = WeightedTrainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()


/tmp/ipython-input-3381214140.py:47: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(


Epoch,Training Loss,Validation Loss,Roc Auc
1,0.742300,0.684958,0.653152
2,0.722300,0.567887,0.827452
3,0.488200,0.422539,0.900054


TrainOutput(global_step=222, training_loss=0.605367542387129, metrics={'train_runtime': 22.1841, 'train_samples_per_second': 79.787, 'train_steps_per_second': 10.007, 'total_flos': 349279925990400.0, 'train_loss': 0.605367542387129, 'epoch': 3.0})

In [ ]:
# %%
val_out = trainer.predict(val_ds)
val_logits = val_out.predictions
val_labels = val_out.label_ids

val_probs = torch.softmax(torch.tensor(val_logits), dim=-1).numpy()[:, 1]
val_preds = (val_probs >= 0.5).astype(int)

print("ROC-AUC:", roc_auc_score(val_labels, val_probs) if len(np.unique(val_labels)) > 1 else "NA")
print(classification_report(val_labels, val_preds, digits=3))


ROC-AUC: 0.9000538793103448
              precision    recall  f1-score   support

           0      0.931     0.931     0.931       116
           1      0.750     0.750     0.750        32

    accuracy                          0.892       148
   macro avg      0.841     0.841     0.841       148
weighted avg      0.892     0.892     0.892       148



In [ ]:
# %%
# ---- Run model on full df (prediction) ----
full_ds = TextClsDataset(df["text"], labels=None, tokenizer=tokenizer, max_len=MAX_LEN)
full_out = trainer.predict(full_ds)
full_logits = full_out.predictions
full_probs = torch.softmax(torch.tensor(full_logits), dim=-1).numpy()[:, 1]

import numpy as np
from sklearn.metrics import precision_recall_curve

def pick_threshold_min_recall(y_true, y_prob, min_recall=0.85):
    precision, recall, thresholds = precision_recall_curve(y_true, y_prob)
    # precision_recall_curve returns thresholds of length n-1
    # recall/precision have length n
    best_t = 0.0
    best_p = -1.0

    for t, p, r in zip(thresholds, precision[1:], recall[1:]):
        if r >= min_recall and p > best_p:
            best_p = p
            best_t = float(t)

    # fallback if target recall unattainable
    if best_p < 0:
        # choose threshold that maximizes recall (usually very low)
        best_t = float(thresholds[np.argmax(recall[1:])])

    return best_t

# --- use on your validation predictions ---
BEST_T = pick_threshold_min_recall(val_labels, val_probs, min_recall=0.80)
print("Chosen threshold (min_recall=0.90):", BEST_T)

df["EVENT_PRED"] = (full_probs >= BEST_T).astype(int)
df["EVENT_PRED"].value_counts()


Chosen threshold (min_recall=0.90): 0.3972659409046173


,count
EVENT_PRED,
0,6985
1,2201


In [ ]:
CB_KEEP = re.compile(
    r"(toplu\s+i[şs]\s+sözleşme|toplu\s+sözleşme|tis\b|"
    r"ücret|zam|sözleşme|protokol|arabulucu|"
    r"grev\s+karar|grev\s+başla|grev\s+çıktı|iş\s+bırak)",
    re.IGNORECASE
)

CB_EXCLUDE = re.compile(
    r"(iş\s+cinayet|kaza|yaralandı|öl(ü|u)m|"
    r"enflasyon|hayat\s+pahalılığı|pahalılık|"
    r"genel\s+değerlendirme|genel\s+yorum|"
    r"işten\s+(çıkar|at|çıkarıl|atıl)|kovuldu|"
    r"sendikalaş|sendika\s+üye|sendika\s+üyeli|"
    r"iş\s+kaz|göçük|gözalt|tutuk|"
    r"dava|mahkeme|ziyaret|dayanışma|anma|basın\s+açıklama)",
    re.IGNORECASE
)

def cb_filter(text: str) -> int:
    t = str(text or "")
    if not CB_KEEP.search(t):
        return 0
    if CB_EXCLUDE.search(t):
        return 0
    return 1

df["EVENT_PRED_CB"] = df.apply(
    lambda r: int(r["EVENT_PRED"] == 1 and cb_filter(r[TEXT_COL]) == 1),
    axis=1
)

print("EVENT_PRED:", int(df["EVENT_PRED"].sum()))
print("EVENT_PRED_CB:", int(df["EVENT_PRED_CB"].sum()))


EVENT_PRED: 2201
EVENT_PRED_CB: 201


In [ ]:
# %%
TR_MONTHS = {
    "ocak": 1, "şubat": 2, "subat": 2, "mart": 3, "nisan": 4, "mayıs": 5, "mayis": 5,
    "haziran": 6, "temmuz": 7, "ağustos": 8, "agustos": 8, "eylül": 9, "eylul": 9,
    "ekim": 10, "kasım": 11, "kasim": 11, "aralık": 12, "aralik": 12
}

def parse_tr_datetime(s):
    if s is None or (isinstance(s, float) and pd.isna(s)):
        return pd.NaT
    s = str(s).strip()
    if not s:
        return pd.NaT

    # Example: "8 Şubat 2024 13:43" (sometimes with extra whitespace)
    m = re.search(r"(\d{1,2})\s+([A-Za-zÇĞİÖŞÜçğıöşü]+)\s+(\d{4})(?:\s+(\d{1,2}):(\d{2}))?", s)
    if not m:
        return pd.NaT

    day = int(m.group(1))
    month_name = m.group(2).lower().replace("ı", "i")  # helps "mayıs/kasım" normalization
    # But keep Turkish chars too:
    month_name = m.group(2).lower()
    month = TR_MONTHS.get(month_name, None)
    if month is None:
        # try ascii fallback
        month = TR_MONTHS.get(month_name.replace("ş","s").replace("ğ","g").replace("ü","u").replace("ö","o").replace("ç","c").replace("ı","i"), None)
    if month is None:
        return pd.NaT

    year = int(m.group(3))
    hh = int(m.group(4)) if m.group(4) else 0
    mm = int(m.group(5)) if m.group(5) else 0

    return pd.Timestamp(year=year, month=month, day=day, hour=hh, minute=mm)

# Create a clean datetime column and use it everywhere downstream
df["DATE_DT"] = df[DATE_COL].apply(parse_tr_datetime)

print("Parsed DATE_DT non-null:", int(df["DATE_DT"].notna().sum()), "/", len(df))
df.loc[df["DATE_DT"].isna(), [DATE_COL]].head(10)


Parsed DATE_DT non-null: 9167 / 9186


,date
7097,konuşmasının devamında şunları söyledi:
7098,NaN
7099,onlar oyalıyor. Biz üretiyoruz
7100,NaN
7101,NaN
7102,NaN
7103,NaN
7104,NaN
7105,NaN
7106,NaN


In [ ]:
from collections import Counter, defaultdict

LEGAL_RE = re.compile(
    r"\b(a\.?ş\.?|aş|anonim|şirketi|şti|ltd|limited|inc|corp|co|holding|sanayi|ticaret|ve)\b",
    flags=re.IGNORECASE
)

# strip role/location words that often attach to firms in news text
ROLE_TAIL_RE = re.compile(
    r"\b("
    r"işçi(leri|ler|lerin|lerin)?|işçilerin|işçileri|"
    r"çalışan(ları|lar|ların)?|"
    r"genel\s+merkez(i)?|genel|merkez(i)?|"
    r"şube(si)?|temsilcilik|"
    r"fabrika(sı|si|da|de|nda|nde)?|"
    r"işletme(si|de|da|nde|nda)?|"
    r"tesis(leri|de|da|nde|nda)?|"
    r"işyeri(nde|ne|ni|de|da)?"
    r")\b",
    flags=re.IGNORECASE
)

def fix_broken_tokenization(text: str) -> str:
    if text is None:
        return ""
    t = str(text)

    # join "d   i   s   k" -> "disk", "i   ş" -> "iş"
    t = re.sub(r"([A-Za-zÇĞİÖŞÜçğıöşü])\s{2,}([A-Za-zÇĞİÖŞÜçğıöşü])", r"\1\2", t)

    # normalize whitespace
    t = re.sub(r"\s+", " ", t).strip()
    return t


df[TITLE_COL] = df[TITLE_COL].apply(fix_broken_tokenization)
df[CONTENT_COL] = df[CONTENT_COL].apply(fix_broken_tokenization)
df[TEXT_COL] = df[TEXT_COL].apply(fix_broken_tokenization)



# %%
# -------- UNION FALLBACK (high-recall) --------

UNION_RE = re.compile(
    r"\b("
    r"birleşik\s+metal(\s+i[şs])?|birlesik\s+metal(\s+i[şs])?|"
    r"türk\s+metal|turk\s+metal|"
    r"genel\s+i[şs]|petrol\s+i[şs]|tek\s+g[ıi]da\s+i[şs]|"
    r"türk\s+i[şs]|turk\s+i[şs]|"
    r"disk|d[ıi]sk|kesk|hak\s+i[şs]|"
    r"tüm\s+bel(\s*-\s*sen|\s+sen)?|tum\s+bel(\s*-\s*sen|\s+sen)?|"
    r"eğitim\s+sen|egitim\s+sen|"
    r"birtek\s+sen"
    r")\b",
    flags=re.IGNORECASE
)

def extract_union_candidates(text: str):
    t = _fold_tr(text)
    found = []
    for m in UNION_RE.finditer(t):
        u = re.sub(r"\s+", " ", m.group(0)).strip()
        u = u.replace("ı", "i")
        found.append(u)
    return list(dict.fromkeys(found))


def looks_broken_tokenized(k: str) -> bool:
    kk = str(k or "")
    if BROKEN_TOKEN_RE.search(kk):
        return True
    # also catch spaced acronyms like "di sk"
    if re.search(r"\b(d|t|k|s)\s+(i|ı)\s*(s|ş)\s*(k|g)\b", kk.lower()):
        return True
    return False


def _fold_tr(s: str) -> str:
    if s is None or (isinstance(s, float) and pd.isna(s)):
        return ""
    s = str(s).strip().lower()
    s = s.replace("’", "'").replace("`", "'")
    s = re.sub(r"\s+", " ", s)
    return s

def canonical_employer(name: str) -> str:
    s = _fold_tr(name)
    if not s:
        return ""
    s = re.sub(r"[^\w\sçğıöşü0-9'-]", " ", s, flags=re.UNICODE)
    s = LEGAL_RE.sub(" ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s


def apply_employer_alias(k: str) -> str:
    # no alias dict
    return canonical_employer(k)

UNION_TERMS = {
    "disk", "türk iş", "turk is", "hak iş", "kesk",
    "türk metal", "metal iş", "birleşik metal iş", "birlesik metal is",
    "genel iş", "petrol iş", "tek gıda iş", "tek gida is",
    "tüm bel sen", "tum bel sen", "tüm bel-sen", "tum bel-sen",
    "birtek sen", "birleşik metal", "birlesik metal", "eğitim sen", "egitim sen",
      "birleşik metal", "birlesik metal",
    "metal iş", "metal is",
    "genel iş", "genel is",
    "türk iş", "turk is",
    "disk genel", "dİsk genel", "d i s k", "di sk"
}
PERSON_TERMS = {
    "hilal tok", "ramis sağlam", "ramis saglam", "özkan atar", "ozkan atar",
    "hayrettin çakmak", "hayrettin cakmak", "cemil tugay",
    "hasret gültekin kozan", "hasret gultekin kozan",
    "sevda karaca", "iskender bayhan", "nazlıer çerik",
    "andac aydın arıduru", "volkan pekal", "özer akdemir",
    "duygu ayber gültekin", "dudu selçuk", "michael roberts", "çerik",
    "murat özveri", "andaç aydın", "ayça pektaş"
}

DROP_IF_CONTAINS = [
    "genel başkan", "genel baskan", "başkanı", "baskani",
    "şube başkanı", "sube baskani",
    "belediyesi", "organize sanayi", "osb",
    "partisi", "chp", "akp", "mhp", "emep",
    "servisi", "muhabir", "haber merkezi", "gazetesi", "ajans",
    # generic wage/strike words are NOT employers:
    "merhaba evrensel", "organize", "çalışma bakanlığı", "şube", "izmir şube",
    "yönetim kurulu", "genel merkez", "genel merkezi",
    "milletvekili", "bakan", "bakanı", "bakanlığı", "bakanligi",
     "valilik", "tbmm", "buyuksehir",
    "parti", "partisi", "dem parti",
]

STARTER_BAD = {
    "yapılan", "yapilan", "önünde", "onunde", "grevdeki",
    "sosyal", "sabah", "aynı", "ayni", "en", "daha", "gece",
    "kamu", "türkiye", "turkiye", "açıklamada", "aciklamada",
    "geçtiğimiz", "gectigimiz", "günlerde", "gunlerde", "çok", "cok"
}

def looks_like_union(k: str) -> bool:
    kk = _fold_tr(k)
    if not kk:
        return False
    # exact OR substring match
    return (kk in UNION_TERMS) or any(u in kk for u in UNION_TERMS)

def looks_like_person(k: str) -> bool:
    kk = _fold_tr(k)
    if not kk:
        return False
    return (kk in PERSON_TERMS) or any(p in kk for p in PERSON_TERMS)


def looks_like_sentence_starter(k: str) -> bool:
    kk = _fold_tr(k)
    toks = kk.split()
    if not toks:
        return True
    if toks[0] in STARTER_BAD:
        return True
    if len(toks) >= 2 and (toks[0] in STARTER_BAD or toks[1] in STARTER_BAD):
        return True
    return False

def looks_like_non_entity_phrase(k: str) -> bool:
    kk = _fold_tr(k)
    if not kk:
        return True
    if kk.endswith(("deki", "daki", "teki", "taki")):
        return True
    if any(w in kk for w in ["açıklamada", "aciklamada", "saatlerinde", "günlerde", "gunlerde"]):
        return True
    return False

# %%
def should_drop_orgish(k: str) -> bool:
    kk = _fold_tr(k)
    if not kk or len(kk) < 3:
        return True
    if looks_like_union(kk) or looks_like_person(kk):
        return True
    for bad in DROP_IF_CONTAINS:
        if bad in kk:
            return True
    return False


In [ ]:
# Triggers (use your existing if you want; this is safe)
ALL_TRIG = re.compile(
    r"(grev(e)? çıktı|grev başladı|iş bırak(tı|ıyor)|üretim(i)? durdur|"
    r"grevde|grev sürüyor|(\d+)\.? ?gün(ü)?nde|"
    r"anlaşma sağlandı|grev bitti|grev sona erdi|protokol imzalandı|"
    r"kabul edildi|reddedildi|imza(landı)?|uzlaş(ma)?|anlaş(ma)?|"
    r"direnişi sürüyor|direniş(i)?)",
    re.IGNORECASE
)

CONTENT_CHARS = 1200
WIN_CHARS = 220
MAX_WINS = 8
MIN_FREQ = 1
MAX_PHRASES = 5000
NGRAM_MIN = 2
NGRAM_MAX = 5

STOP = set("""
ve veya ile için gibi üzere da de ki mi mı mu mü
işçi işçileri grev grevi direniş direnişi eylem açıklama basın
sendika sendikası işçilerden işçilerin mücadele talep sözleşme toplu iş emekçi
emekçiler emekçileri örgütlü ama fakat lakin çünkü
""".split())

def clean_for_phrase_mining(text: str) -> str:
    text = (text or "")
    text = text.replace("\u00A0", " ").replace("’","'").replace("`","'")
    text = re.sub(r"\s+", " ", text).strip()
    return text

def tokenize_simple(text: str):
    return re.findall(r"[A-Za-zÇĞİÖŞÜçğıöşü0-9]+", text)

def is_titlecase_like(tok: str) -> bool:
    if len(tok) < 2:
        return False
    if tok.isupper() and len(tok) >= 2:
        return True
    return tok[0].isupper() and any(c.islower() for c in tok[1:])

def trigger_windows_text(title: str, content: str, win_chars=WIN_CHARS, max_wins=MAX_WINS):
    t = clean_for_phrase_mining(title)
    c = clean_for_phrase_mining(content)[:CONTENT_CHARS]
    text = f"{t} {c}".strip()
    wins = []
    for m in ALL_TRIG.finditer(text):
        a = max(0, m.start() - win_chars)
        b = min(len(text), m.end() + win_chars)
        wins.append(text[a:b])
        if len(wins) >= max_wins:
            break
    return wins

phrase_counts = Counter()

# Mine only from predicted relevant docs (speeds up + reduces boilerplate)
mask_mine = df["EVENT_PRED_CB"] == 1

for t, c in zip(df.loc[mask_mine, TITLE_COL].astype(str), df.loc[mask_mine, CONTENT_COL].astype(str)):
    wins = trigger_windows_text(t, c)
    if not wins:
        continue
    for w in wins:
        toks = tokenize_simple(w)
        flags = [is_titlecase_like(tok) for tok in toks]

        i = 0
        while i < len(toks):
            if not flags[i]:
                i += 1
                continue
            j = i
            while j < len(toks) and flags[j]:
                j += 1

            span = toks[i:j]
            for n in range(NGRAM_MIN, NGRAM_MAX + 1):
                for k in range(0, len(span) - n + 1):
                    ng = span[k:k+n]
                    ng_l = [w.lower() for w in ng]
                    if any(w in STOP for w in ng_l):
                        continue
                    if all(w.isdigit() for w in ng):
                        continue
                    phrase = " ".join(ng)
                    if any(len(w) == 1 for w in ng):
                      continue
                    phrase_counts[phrase] += 1

            i = j

candidates = [(p, cnt) for p, cnt in phrase_counts.items() if cnt >= MIN_FREQ]
candidates.sort(key=lambda x: x[1], reverse=True)

firm_phrases = [p for p, cnt in candidates[:MAX_PHRASES]]

print("Candidate phrases:", len(candidates))
print("firm_phrases kept:", len(firm_phrases))
print("Top 30 candidates:")
for p, cnt in candidates[:30]:
    print(cnt, "-", p)



Candidate phrases: 1380
firm_phrases kept: 1380
Top 30 candidates:
47 - Genel İş
43 - Özçelik İş
33 - Birleşik Metal
33 - Metal İş
33 - Birleşik Metal İş
27 - Şube Başkanı
17 - Başkanı Mehmet
17 - DİSK Genel
16 - DİSK Genel İş
15 - Green Transfo
14 - MMK Metalurji
14 - Genel Başkanı
14 - Grid Solutions
13 - İş İstanbul
13 - Türk İş
12 - İskenderun Şube
12 - İskenderun Şube Başkanı
12 - Şube Başkanı Mehmet
12 - İskenderun Şube Başkanı Mehmet
11 - Mehmet Güngör
11 - Türk Metal
11 - GE Grid
10 - Başkanı Mehmet Güngör
10 - Şube Başkanı Mehmet Güngör
10 - İskenderun Şube Başkanı Mehmet Güngör
10 - İstanbul Anadolu
10 - Genel İş İstanbul
10 - İş İstanbul Anadolu
10 - Genel İş İstanbul Anadolu
10 - Uluslararası Liman


In [ ]:
import spacy
from spacy.pipeline import EntityRuler

nlp_ruler = spacy.blank("tr")
ruler = nlp_ruler.add_pipe("entity_ruler")

patterns = [{"label": "ORG", "pattern": p} for p in firm_phrases]
ruler.add_patterns(patterns)

def extract_org_keys_ruler(title: str, content: str):
    txt = f"{title} {content[:CONTENT_CHARS]}"
    doc = nlp_ruler(txt)
    orgs = []
    for ent in doc.ents:
        if ent.label_ != "ORG":
            continue
        k = canonical_employer(ent.text)
        if not k:
            continue
        orgs.append(k)
    # dedup preserve order
    return list(dict.fromkeys(orgs))

df["ORG_KEYS"] = [
    extract_org_keys_ruler(t, c)
    for t, c in zip(df[TITLE_COL].astype(str), df[CONTENT_COL].astype(str))
]

# filter
def is_good_org_key(k: str) -> bool:
    k2 = canonical_employer(k)
    if not k2:
        return False
    if len(k2.split()) < 2:
        return False
    if len(k2) < 6:
        return False
    if should_drop_orgish(k2):
        return False
    return True

df["ORG_KEYS_FILTERED"] = df["ORG_KEYS"].apply(lambda ks: [canonical_employer(k) for k in (ks or []) if is_good_org_key(k)])

print("Docs with >=1 ORG_KEY before:", int((df["ORG_KEYS"].apply(lambda x: len(x or [])) > 0).sum()))
print("Docs with >=1 ORG_KEY after :", int((df["ORG_KEYS_FILTERED"].apply(lambda x: len(x or [])) > 0).sum()))



Docs with >=1 ORG_KEY before: 5931
Docs with >=1 ORG_KEY after : 3085


In [ ]:
P_FACILITY = re.compile(
    r"(?P<name>[A-ZÇĞİÖŞÜ][\w’'-.]+(?:\s+[A-ZÇĞİÖŞÜ][\w’'-.]+){0,6})\s+"
    r"(fabrika(sı|sinda|sında|da|de|nda|nde)?|işyeri(nde|ne|ni)?|tesis(leri|de|da|nde|nda)?)",
    re.UNICODE
)
P_WORKERS = re.compile(
    r"(?P<name>[A-ZÇĞİÖŞÜ][\w’'-.]+(?:\s+[A-ZÇĞİÖŞÜ][\w’'-.]+){0,6})\s+işçi(leri|ler)?",
    re.UNICODE
)

def _as_list(x):
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return []
    if isinstance(x, list):
        return [str(i).strip() for i in x if str(i).strip()]
    s = str(x).strip()
    if not s:
        return []
    return [p.strip() for p in re.split(r"[;|,]\s*", s) if p.strip()]

def trigger_windows(text: str, window_chars: int = 180, max_wins: int = 6):
    t = text or ""
    wins = []
    for m in ALL_TRIG.finditer(t):
        a = max(0, m.start() - window_chars)
        b = min(len(t), m.end() + window_chars)
        wins.append(t[a:b])
        if len(wins) >= max_wins:
            break
    return wins

# %%
def extract_employer_candidates_from_text(title: str, content: str):
    txt = (str(title or "") + " " + str(content or "")).strip()
    wins = trigger_windows(txt, window_chars=180, max_wins=6)

    emp, uni = [], []
    for w in wins:
        for pat in (P_FACILITY, P_WORKERS):
            for m in pat.finditer(w):
                raw = m.group("name").strip()
                k = canonical_employer(raw)
                if not k:
                    continue

                if looks_like_union(k):
                    uni.append(k)
                    continue
                if looks_like_person(k):
                    continue
                if should_drop_orgish(k):
                    continue

                emp.append(k)

    def dedup(seq):
        seen = set()
        out = []
        for x in seq:
            if x and x not in seen:
                seen.add(x)
                out.append(x)
        return out

    return dedup(emp), dedup(uni)

# %%
def build_employer_and_union_keys(df_in, org_col="ORG_KEYS_FILTERED", max_emp_per_doc=3):
    df_out = df_in.copy()
    employer_keys = []
    union_keys = []

    for t, c, orgs in zip(
        df_out["title"].astype(str),
        df_out["content"].astype(str),
        df_out[org_col] if org_col in df_out.columns else [None] * len(df_out)
    ):
        org_list = _as_list(orgs)

        emp_from_org, uni_from_org = [], []
        for o in org_list:
            k = apply_employer_alias(o)
            if not k:
                continue
            if looks_like_union(k):
                uni_from_org.append(k)
                continue
            if looks_like_person(k):
                continue
            if should_drop_orgish(k):
                continue
            emp_from_org.append(k)

        emp_mined, uni_mined = extract_employer_candidates_from_text(t, c)
        uni_text = extract_union_candidates(f"{t} {c}")

        def dedup(seq):
            seen = set()
            out = []
            for x in seq:
                if x and x not in seen:
                    seen.add(x)
                    out.append(x)
            return out

        emp = dedup(emp_from_org + emp_mined)[:max_emp_per_doc]
        uni = dedup(uni_from_org + uni_mined + uni_text)

        employer_keys.append(emp)
        union_keys.append(uni)

    df_out["EMPLOYER_KEYS"] = employer_keys
    df_out["UNION_KEYS"] = union_keys
    return df_out


df_linked = df.copy()
df_linked = build_employer_and_union_keys(df_linked, org_col="ORG_KEYS_FILTERED", max_emp_per_doc=3)


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel


EMB_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
emb_tokenizer = AutoTokenizer.from_pretrained(EMB_MODEL_NAME)
emb_model = AutoModel.from_pretrained(EMB_MODEL_NAME).to("cuda" if torch.cuda.is_available() else "cpu")
emb_model.eval()

def mean_pool(last_hidden, attention_mask):
    mask = attention_mask.unsqueeze(-1).expand(last_hidden.size()).float()
    summed = (last_hidden * mask).sum(dim=1)
    counts = mask.sum(dim=1).clamp(min=1e-9)
    return summed / counts

@torch.no_grad()
def encode_texts(texts, batch_size=32, max_len=256):
    dev = "cuda" if torch.cuda.is_available() else "cpu"
    vecs = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        enc = emb_tokenizer(
            batch,
            truncation=True,
            max_length=max_len,
            padding=True,
            return_tensors="pt"
        ).to(dev)
        out = emb_model(**enc)
        v = mean_pool(out.last_hidden_state, enc["attention_mask"])
        v = torch.nn.functional.normalize(v, p=2, dim=1)
        vecs.append(v.detach().cpu().numpy())
    return np.vstack(vecs)

def connected_components(n, edges):
    parent = list(range(n))
    rank = [0]*n

    def find(x):
        while parent[x] != x:
            parent[x] = parent[parent[x]]
            x = parent[x]
        return x

    def union(a, b):
        ra, rb = find(a), find(b)
        if ra == rb:
            return
        if rank[ra] < rank[rb]:
            parent[ra] = rb
        elif rank[ra] > rank[rb]:
            parent[rb] = ra
        else:
            parent[rb] = ra
            rank[ra] += 1

    for a, b in edges:
        union(a, b)

    comps = defaultdict(list)
    for i in range(n):
        comps[find(i)].append(i)
    return list(comps.values())



tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

In [ ]:
def build_rare_tokens(titles, min_len=4, max_df=0.05):
    toks_all = []
    for t in titles:
        toks = re.findall(r"[A-Za-zÇĞİÖŞÜçğıöşü]+", str(t).lower())
        toks = [x for x in toks if len(x) >= min_len]
        toks_all.extend(set(toks))
    c = Counter(toks_all)
    n = len(titles)
    return {tok for tok, dfreq in c.items() if (dfreq / max(n,1)) <= max_df}

rare_tokens = build_rare_tokens(df_linked[TITLE_COL].tolist(), min_len=4, max_df=0.05)

def rare_title_tokens(title, min_len=4):
    toks = re.findall(r"[A-Za-zÇĞİÖŞÜçğıöşü]+", str(title).lower())
    return {t for t in toks if len(t) >= min_len and t in rare_tokens}



In [ ]:
def _safe_days_diff(d1, d2):
    if pd.isna(d1) or pd.isna(d2):
        return None
    try:
        return abs((pd.to_datetime(d1) - pd.to_datetime(d2)).days)
    except Exception:
        return None

def _time_decay(days_diff, tau_days=25.0):
    if days_diff is None:
        return 1.0
    return math.exp(-float(days_diff) / float(tau_days))

def assign_event_ids_hybrid(
    df_in,
    rel_flag_col="EVENT_PRED_CB",
    date_col=DATE_COL,
    employer_col="EMPLOYER_KEYS",
    sim_short=0.945,
    sim_emp=0.965,
    tau_days=40.0,
    max_rel=6000,
    EMP_MAX_BUCKET=40,
    TITLE_OVERLAP_K=2,
    MIN_CLUSTER_SIZE=2,
    DROP_EMPTY_EMPLOYER_CLUSTERS=True,
):
    df_out = df_in.copy()

    if EVENT_ID_COL not in df_out.columns:
        df_out[EVENT_ID_COL] = np.nan

    df_out[EVENT_ID_COL] = df_out[EVENT_ID_COL].where(df_out[EVENT_ID_COL].notna(), np.nan)
    df_out[EVENT_ID_COL] = df_out[EVENT_ID_COL].apply(lambda x: str(x).strip() if not pd.isna(x) else np.nan)

    rel = df_out[df_out[rel_flag_col] == 1].copy()
    if len(rel) == 0:
        print(f"No rows where {rel_flag_col} == 1.")
        return df_out
    if len(rel) > max_rel:
        print(f"Too many relevant rows ({len(rel)}).")
        return df_out

    rel["_has_date"] = rel[date_col].notna()
    rel = rel.sort_values(by=[date_col, "_has_date"], ascending=[True, False])

    idx = list(rel.index)
    n = len(idx)

    E = encode_texts(rel[TEXT_COL].tolist(), batch_size=32 if torch.cuda.is_available() else 8, max_len=256)

    title_tok_sets = [rare_title_tokens(t) for t in rel[TITLE_COL].tolist()]
    rel_dates = rel[date_col].tolist()

    emp_to_pos = defaultdict(list)
    rel_emp_lists = rel[employer_col].tolist() if employer_col in rel.columns else [None] * n
    for pos, keys in enumerate(rel_emp_lists):
        for k in (keys or []):
            if k:
                emp_to_pos[k].append(pos)

    edges = set()

    # A) Employer-bucket edges
    for emp, positions in emp_to_pos.items():
        if len(positions) <= 1:
            continue
        if len(positions) > EMP_MAX_BUCKET:
            continue
        for a_i in range(len(positions)):
            i = positions[a_i]
            for a_j in range(a_i + 1, len(positions)):
                j = positions[a_j]
                base_sim = float(np.dot(E[i], E[j]))
                dd = _safe_days_diff(rel_dates[i], rel_dates[j])
                sim = base_sim * _time_decay(dd, tau_days=tau_days)
                if sim >= sim_emp:
                    if len(title_tok_sets[i] & title_tok_sets[j]) >= TITLE_OVERLAP_K:
                        edges.add((min(i, j), max(i, j)))

    # B) Non-employer edges
    for i in range(n):
        for j in range(i + 1, n):
            base_sim = float(np.dot(E[i], E[j]))
            dd = _safe_days_diff(rel_dates[i], rel_dates[j])
            sim = base_sim * _time_decay(dd, tau_days=tau_days)
            if sim >= sim_short:
                if len(title_tok_sets[i] & title_tok_sets[j]) >= TITLE_OVERLAP_K:
                    edges.add((i, j))

    comps = connected_components(n, list(edges))
    clusters = [[idx[pos] for pos in comp] for comp in comps]

    # Filter phantom clusters
    filtered = []
    for cluster in clusters:
        if len(cluster) < MIN_CLUSTER_SIZE:
            continue
        if DROP_EMPTY_EMPLOYER_CLUSTERS and employer_col in df_out.columns:
            emp_nonempty = df_out.loc[cluster, employer_col].apply(lambda x: isinstance(x, list) and len(x) > 0).sum()
            if int(emp_nonempty) == 0:
                continue
        filtered.append(cluster)
    clusters = filtered

    # Assign EVENT_ID
    new_counter = 1
    for cluster in clusters:
        existing = df_out.loc[cluster, EVENT_ID_COL].dropna()
        if len(existing) > 0:
            chosen = existing.value_counts().idxmax()
        else:
            chosen = f"EV{new_counter:06d}"
            new_counter += 1
        df_out.loc[cluster, EVENT_ID_COL] = chosen

    return df_out

df_linked = assign_event_ids_hybrid(
    df_linked,
    rel_flag_col="EVENT_PRED_CB",
    date_col=DATE_COL,
    employer_col="EMPLOYER_KEYS",
    sim_short=0.90,
    sim_emp=0.94,
    tau_days=35.0,
    EMP_MAX_BUCKET=40,
    TITLE_OVERLAP_K=1,
    MIN_CLUSTER_SIZE=2,
    DROP_EMPTY_EMPLOYER_CLUSTERS=True
)

mask_rel = df_linked["EVENT_PRED_CB"] == 1
print("Pred-relevant (CB/Wage):", int(mask_rel.sum()))
print("Unique EVENT_ID among predicted relevant:", int(df_linked.loc[mask_rel, EVENT_ID].nunique()))
print(df_linked.loc[mask_rel].groupby("EVENT_ID").size().describe())



Pred-relevant (CB/Wage): 201
Unique EVENT_ID among predicted relevant: 14
count    14.000000
mean      1.500000
std       0.518875
min       1.000000
25%       1.000000
50%       1.500000
75%       2.000000
max       2.000000
dtype: float64


In [ ]:
def firm_split_event_ids(df_in, rel_flag_col="EVENT_PRED_CB", event_col="EVENT_ID", employer_col="EMPLOYER_KEYS"):
    df_out = df_in.copy()
    df_out["EVENT_ID_FIRM"] = np.nan

    rel = df_out[df_out[rel_flag_col] == 1].copy()
    for ev, g in rel.groupby(event_col):
        # assign per employer; if no employer, keep one bucket
        for idx_row in g.index:
            emps = df_out.at[idx_row, employer_col]
            if not isinstance(emps, list) or len(emps) == 0:
                df_out.at[idx_row, "EVENT_ID_FIRM"] = f"{ev}_F0"
            else:
                # pick first employer as primary key for firm-level split
                primary = emps[0]
                df_out.at[idx_row, "EVENT_ID_FIRM"] = f"{ev}_{primary}"
    return df_out

df_linked = firm_split_event_ids(df_linked, rel_flag_col="EVENT_PRED_CB", event_col="EVENT_ID", employer_col="EMPLOYER_KEYS")



/tmp/ipython-input-417053195.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'E0013_i skenderun yolbulan' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_out.at[idx_row, "EVENT_ID_FIRM"] = f"{ev}_{primary}"


In [ ]:
# %%
# --- Export preflight: make sure df_linked has the columns we want ---

# 1) If EVENT_PROB exists on df (prediction ran there), bring it over by index
if "EVENT_PROB" not in df_linked.columns:
    if "EVENT_PROB" in df.columns:
        df_linked = df_linked.join(df[["EVENT_PROB"]], how="left")
    else:
        df_linked["EVENT_PROB"] = np.nan

# 2) Same idea for EVENT_PRED / EVENT_PRED_CB if you computed on df not df_linked
for col in ["EVENT_PRED", "EVENT_PRED_CB"]:
    if col not in df_linked.columns:
        if col in df.columns:
            df_linked = df_linked.join(df[[col]], how="left")
        else:
            df_linked[col] = 0

# 3) Manual column may be missing / sparse
if MANUAL_COL not in df_linked.columns:
    df_linked[MANUAL_COL] = np.nan

# Quick check
print("Missing among export-critical cols:",
      [c for c in ["EVENT_PROB","EVENT_PRED","EVENT_PRED_CB",MANUAL_COL] if c not in df_linked.columns])


Missing among export-critical cols: []


In [ ]:
from openpyxl import Workbook

DATE_COL = "DATE_DT"

if "EVENT_PROB" not in df_linked.columns and "EVENT_PROB" in df.columns:
    df_linked = df_linked.join(df[["EVENT_PROB"]], how="left")

def list_to_str(x):
    if isinstance(x, list):
        return "; ".join([str(i) for i in x if str(i).strip()])
    if pd.isna(x):
        return ""
    return str(x)

def flatten_unique_keys(series_of_lists):
    s = set()
    for ks in series_of_lists:
        if ks is None or (isinstance(ks, float) and pd.isna(ks)):
            continue
        if isinstance(ks, list):
            for k in ks:
                k = str(k).strip()
                if k:
                    s.add(k)
        else:
            for k in str(ks).split(";"):
                k = k.strip()
                if k:
                    s.add(k)
    return "; ".join(sorted(s))

mask = df_linked["EVENT_PRED_CB"] == 1

events_firm = (
    df_linked.loc[mask]
    .groupby("EVENT_ID_FIRM", dropna=False)
    .agg(
        start=("DATE_DT", "min"),
        end=("DATE_DT", "max"),
        duration=("DATE_DT", lambda x: (pd.to_datetime(x).max() - pd.to_datetime(x).min()).days + 1 if x.notna().any() else ""),
        n_articles=(TITLE_COL, "count"),
        firms=("ORG_KEYS_FILTERED", flatten_unique_keys),
        employers=("EMPLOYER_KEYS", flatten_unique_keys),
        unions=("UNION_KEYS", flatten_unique_keys),
    )
    .reset_index()
)

for c in ["start", "end"]:
    events_firm[c] = pd.to_datetime(events_firm[c], errors="coerce")

wb = Workbook()
ws1 = wb.active
ws1.title = "Firm_Level_Strikes"
ws1.append(list(events_firm.columns))

for _, row in events_firm.iterrows():
    ws1.append([list_to_str(v) for v in row.tolist()])

ws2 = wb.create_sheet("Articles_By_Firm_Event")
cols = ["EVENT_ID_FIRM", "DATE_DT", TITLE_COL, "ORG_KEYS_FILTERED", "EMPLOYER_KEYS", "UNION_KEYS", "EVENT_ID", MANUAL_COL, "EVENT_PRED", "EVENT_PRED_CB", "EVENT_PROB"]
ws2.append(cols)

tmp = df_linked.loc[mask, cols].copy()
tmp[DATE_COL] = pd.to_datetime(tmp[DATE_COL], errors="coerce")
for col in ["ORG_KEYS_FILTERED", "EMPLOYER_KEYS", "UNION_KEYS"]:
    tmp[col] = tmp[col].apply(list_to_str)

for _, r in tmp.iterrows():
    ws2.append([list_to_str(v) for v in r.tolist()])

path = "firm_level_strikes_trigger_window.xlsx"
wb.save(path)
print("Saved:", path)



Saved: firm_level_strikes_trigger_window.xlsx


In [ ]:
def top30_listcol(df_in, col, mask):
    c = Counter()
    for ks in df_in.loc[mask, col]:
        if ks is None or (isinstance(ks, float) and pd.isna(ks)):
            continue
        if isinstance(ks, list):
            for k in ks:
                k = str(k).strip()
                if k:
                    c[k] += 1
        else:
            for k in str(ks).split(";"):
                k = k.strip()
                if k:
                    c[k] += 1
    return c.most_common(30)

print("Top 30 ORG_KEYS_FILTERED:")
for k, v in top30_listcol(df_linked, "ORG_KEYS_FILTERED", df_linked["EVENT_PRED_CB"]==1):
    print(v, "-", k)

print("\nTop 30 EMPLOYER_KEYS:")
for k, v in top30_listcol(df_linked, "EMPLOYER_KEYS", df_linked["EVENT_PRED_CB"]==1):
    print(v, "-", k)

Top 30 ORG_KEYS_FILTERED:
11 - i ş i zmir
6 - i ş sözleşmesi
6 - i şçi sendi ka servi si
6 - hitachi energy
5 - tüm bel
5 - i ş i stanbul anadolu yakası
5 - i zmir büyükşehir
5 - schneider elektrik
5 - arıtaş kriyojenik
5 - türkiye metal sanayicileri
4 - sen i zmir
4 - i ş sözleşmeleri
4 - green transfo
4 - ge grid solutions
3 - i ş i stanbul
3 - grid solutions
3 - schneider electric
3 - i ş aliağa
3 - birleşik kamu
3 - i ş sendikasına
3 - ege üniversitesi hastanesindeki sağlık
2 - yolbulan metal
2 - adaletin i ş
2 - i zel gözde meydan
2 - ravago enplast
2 - grevdeki sarar
2 - körfez kıyısı
2 - uluslararası liman i şçileri
2 - öz büro i ş
2 - pamukkale üniversitesi

Top 30 EMPLOYER_KEYS:
11 - i ş i zmir
6 - hitachi energy
5 - tüm bel
5 - i ş i stanbul anadolu yakası
5 - i zmir büyükşehir
5 - türkiye metal sanayicileri
4 - sen i zmir
4 - i şçi sendi ka servi si
4 - bir
4 - i ş sözleşmeleri
4 - ge grid solutions
3 - belediye
3 - i ş i stanbul
3 - grid solutions
3 - schneider elektrik
3 -

In [ ]:
cnt = Counter()
for ks in df["ORG_KEYS_FILTERED"]:
    for k in ks:
        cnt[k] += 1
print("Top 30 ORG_KEYS_FILTERED:")
for k,v in cnt.most_common(30):
    print(v, "-", k)

Top 30 ORG_KEYS_FILTERED:
216 - sosyal güvenlik
212 - i ş genel
184 - i ş i zmir
181 - tüm bel
165 - i şçi sendi ka servi si
155 - i zmir büyükşehir
120 - i ş güvenliği
110 - sen i zmir
102 - i şçi sağlığı
86 - i ş sözleşmesi
77 - i ş sendikasının
69 - i ş gebze
64 - temel conta
55 - öz i plik
55 - birleşik kamu
54 - i ş sendikasına
52 - murat uysal
51 - başkan yardımcısı
49 - i ş yeri temsilcisi
43 - sendi ka servi si
43 - çayırhan termik santrali
43 - arıtaş kriyojenik
43 - green transfo
42 - genel i ş
42 - adaletin i ş yüzü
42 - sosyal demokrat kamu i şverenleri
42 - i ş i stanbul anadolu yakası
41 - sen genel
41 - mehmet türkmen
40 - i ş i stanbul


In [ ]:
check = df_linked["EVENT_PRED_CB"] == 1
print("Pred relevant:", check.sum())
print("Unique events:", df_linked.loc[check, "EVENT_ID"].nunique())

Pred relevant: 201
Unique events: 14


In [ ]:
check2 = df_linked["EVENT_PRED_CB"] == 1
empty_org = df_linked.loc[check2, "ORG_KEYS_FILTERED"].apply(lambda x: len(x or []) == 0).mean()
print("Share of CB-relevant with empty ORG:", empty_org)


Share of CB-relevant with empty ORG: 0.3383084577114428


In [ ]:
mask = df_linked["EVENT_PRED_CB"] == 1

def nlist(x):
    return len(x or [])

print("CB relevant:", int(mask.sum()))
print("empty ORG_KEYS_FILTERED:", (df_linked.loc[mask, "ORG_KEYS_FILTERED"].apply(nlist) == 0).mean())
print("empty EMPLOYER_KEYS:", (df_linked.loc[mask, "EMPLOYER_KEYS"].apply(nlist) == 0).mean())
print("empty UNION_KEYS:", (df_linked.loc[mask, "UNION_KEYS"].apply(nlist) == 0).mean())

CB relevant: 201
empty ORG_KEYS_FILTERED: 0.3383084577114428
empty EMPLOYER_KEYS: 0.2935323383084577
empty UNION_KEYS: 0.8407960199004975
